In [ ]:
import pandas as pd
import seaborn as sns
import jsonlines
import os
import matplotlib.pyplot as plt

from src.toolkit.process_results import extract_results

In [ ]:
dataframes = extract_results("/DATA/ocl_survey/er_split_cifar100_20_2000")

In [ ]:
df_continual = dataframes["continual"]
df_training = dataframes["training"]
df_probing = dataframes["probing"]

In [ ]:
print(df_training["Top1_Acc_Stream/eval_phase/test_stream/Task000"].dropna())

In [ ]:
# This computes the test accuracy curves, one point after each experience
# the frame is reindexed so that it can match the curve of probing

final_test_acc = df_training.groupby("mb_index").agg(["mean", "std"])["Top1_Acc_Stream/eval_phase/test_stream/Task000"]
final_test_acc["training_task"] = final_test_acc.index.argsort()
final_test_acc = final_test_acc.fillna(0)

final_probing = df_probing.groupby("mb_index").agg(["mean", "std"])["Top1_Acc_Stream/eval_phase/test_stream/Task000"]
final_probing["training_task"] = final_probing.index.argsort()
final_probing = final_probing.fillna(0)

sns.lineplot(data=final_probing, x="training_task", y="mean", label="ER (Probed)")

plt.fill_between(final_probing['training_task'], final_probing['mean'] - final_probing['std'], final_probing['mean'] + final_probing['std'], alpha=0.2)

sns.lineplot(data=final_test_acc, x="training_task", y="mean", label="ER")

plt.fill_between(final_test_acc['training_task'], final_test_acc['mean'] - final_test_acc['std'], final_test_acc['mean'] + final_test_acc['std'], alpha=0.2)

plt.ylabel("Average Accuracy")
plt.xlabel("Training Task")
plt.grid()
plt.ylim(0.1, 0.8)

In [ ]:
#sns.lineplot(data=df_training, x="mb_index", y="Top1_Acc_Stream/eval_phase/test_stream/Task000", errorbar="sd")
sns.lineplot(data=df_probing, x="mb_index", y="Top1_Acc_Stream/eval_phase/test_stream/Task000", errorbar="sd")